In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
import time
import random
import pdb


### Make a list of the urls of the categories in the forum

In [3]:
user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
html_page = requests.get('https://curltalk.naturallycurly.com/', headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser')
link_list_forum_top = []
for link in soup.find_all('a'):
    link_list_forum_top.append(link.get('href'))
categories = [s for s in link_list_forum_top if (("categories" in s) and ('https' in s))]
categories[0:]

['https://curltalk.naturallycurly.com/categories/announcements',
 'https://curltalk.naturallycurly.com/categories/newbies-info',
 'https://curltalk.naturallycurly.com/categories/introductions',
 'https://curltalk.naturallycurly.com/categories/curlmart-news',
 'https://curltalk.naturallycurly.com/categories/feedback',
 'https://curltalk.naturallycurly.com/categories/c2',
 'https://curltalk.naturallycurly.com/categories/c3',
 'https://curltalk.naturallycurly.com/categories/c4',
 'https://curltalk.naturallycurly.com/categories/general-discussion-about-curly-hair',
 'https://curltalk.naturallycurly.com/categories/salon-stylist-chatter',
 'https://curltalk.naturallycurly.com/categories/transitioning',
 'https://curltalk.naturallycurly.com/categories/thin-thinning-hair',
 'https://curltalk.naturallycurly.com/categories/readers-product-reviews',
 'https://curltalk.naturallycurly.com/categories/hall-of-shame-discussion',
 'https://curltalk.naturallycurly.com/categories/hall-of-fame-discussion'

#### Get the discussion topic urls for the first x pages of the category

In [74]:
# %%time

link_listdiscussion = []
for i in range(0, 100):
    url = f'{categories[29]}/p{i}'
    print(url)
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
     
    # Check status code for an appropriate response fromt the API
    status = html_page.status_code
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        #break
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    page1_of_discussion = []
    #time.sleep(random.randint(20,45))
    for link in soup.find_all('a'):
        link_listdiscussion.append(link.get('href'))
        topics = [s for s in link_listdiscussion if (("/discussion" in s) and ('https' in s))]

print(len(topics))     
# Loop through all the topics found for each catagory        
list_for_mongo = []
count = 1
for topic in topics:
    url2 = topic
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page2 = requests.get(url2, headers = {'User-Agent': user_agent} )
    
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        #break
        
    soup2 = BeautifulSoup(html_page2.content, 'html.parser')
    signatures = soup2.find_all('div', class_="Signature UserSignature userContent")
    
    for i in range(0,len(signatures)):
        sig = {}
        sig['signature'] = soup2.find_all('div', class_="Signature UserSignature userContent")[i].get_text()
        list_for_mongo.append(sig)
#     time.sleep(random.randint(1,5))
    print(count)
    count += 1
    

https://curltalk.naturallycurly.com/categories/growing-out/p0
https://curltalk.naturallycurly.com/categories/growing-out/p1
https://curltalk.naturallycurly.com/categories/growing-out/p2
https://curltalk.naturallycurly.com/categories/growing-out/p3
https://curltalk.naturallycurly.com/categories/growing-out/p4
https://curltalk.naturallycurly.com/categories/growing-out/p5
https://curltalk.naturallycurly.com/categories/growing-out/p6
https://curltalk.naturallycurly.com/categories/growing-out/p7
https://curltalk.naturallycurly.com/categories/growing-out/p8
https://curltalk.naturallycurly.com/categories/growing-out/p9
https://curltalk.naturallycurly.com/categories/growing-out/p10
https://curltalk.naturallycurly.com/categories/growing-out/p11
https://curltalk.naturallycurly.com/categories/growing-out/p12
https://curltalk.naturallycurly.com/categories/growing-out/p13
https://curltalk.naturallycurly.com/categories/growing-out/p14
https://curltalk.naturallycurly.com/categories/growing-out/p15
ht

https://curltalk.naturallycurly.com/categories/growing-out/p96
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/growing-out/p97
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/growing-out/p98
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/growing-out/p99
Error improper response code. Code is 404
1304
Error improper response code. Code is 404
1
Error improper response code. Code is 404
2
Error improper response code. Code is 404
3
Error improper response code. Code is 404
4
Error improper response code. Code is 404
5
Error improper response code. Code is 404
6
Error improper response code. Code is 404
7
Error improper response code. Code is 404
8
Error improper response code. Code is 404
9
Error improper response code. Code is 404
10
Error improper response code. Code is 404
11
Error improper response code. Code is 404
12
Error improper response code. Code is 404
13

Error improper response code. Code is 404
173
Error improper response code. Code is 404
174
Error improper response code. Code is 404
175
Error improper response code. Code is 404
176
Error improper response code. Code is 404
177
Error improper response code. Code is 404
178
Error improper response code. Code is 404
179
Error improper response code. Code is 404
180
Error improper response code. Code is 404
181
Error improper response code. Code is 404
182
Error improper response code. Code is 404
183
Error improper response code. Code is 404
184
Error improper response code. Code is 404
185
Error improper response code. Code is 404
186
Error improper response code. Code is 404
187
Error improper response code. Code is 404
188
Error improper response code. Code is 404
189
Error improper response code. Code is 404
190
Error improper response code. Code is 404
191
Error improper response code. Code is 404
192
Error improper response code. Code is 404
193
Error improper response code. Code

Error improper response code. Code is 404
352
Error improper response code. Code is 404
353
Error improper response code. Code is 404
354
Error improper response code. Code is 404
355
Error improper response code. Code is 404
356
Error improper response code. Code is 404
357
Error improper response code. Code is 404
358
Error improper response code. Code is 404
359
Error improper response code. Code is 404
360
Error improper response code. Code is 404
361
Error improper response code. Code is 404
362
Error improper response code. Code is 404
363
Error improper response code. Code is 404
364
Error improper response code. Code is 404
365
Error improper response code. Code is 404
366
Error improper response code. Code is 404
367
Error improper response code. Code is 404
368
Error improper response code. Code is 404
369
Error improper response code. Code is 404
370
Error improper response code. Code is 404
371
Error improper response code. Code is 404
372
Error improper response code. Code

Error improper response code. Code is 404
531
Error improper response code. Code is 404
532
Error improper response code. Code is 404
533
Error improper response code. Code is 404
534
Error improper response code. Code is 404
535
Error improper response code. Code is 404
536
Error improper response code. Code is 404
537
Error improper response code. Code is 404
538
Error improper response code. Code is 404
539
Error improper response code. Code is 404
540
Error improper response code. Code is 404
541
Error improper response code. Code is 404
542
Error improper response code. Code is 404
543
Error improper response code. Code is 404
544
Error improper response code. Code is 404
545
Error improper response code. Code is 404
546
Error improper response code. Code is 404
547
Error improper response code. Code is 404
548
Error improper response code. Code is 404
549
Error improper response code. Code is 404
550
Error improper response code. Code is 404
551
Error improper response code. Code

709
Error improper response code. Code is 404
710
Error improper response code. Code is 404
711
Error improper response code. Code is 404
712
Error improper response code. Code is 404
713
Error improper response code. Code is 404
714
Error improper response code. Code is 404
715
Error improper response code. Code is 404
716
Error improper response code. Code is 404
717
Error improper response code. Code is 404
718
Error improper response code. Code is 404
719
Error improper response code. Code is 404
720
Error improper response code. Code is 404
721
Error improper response code. Code is 404
722
Error improper response code. Code is 404
723
Error improper response code. Code is 404
724
Error improper response code. Code is 404
725
Error improper response code. Code is 404
726
Error improper response code. Code is 404
727
Error improper response code. Code is 404
728
Error improper response code. Code is 404
729
Error improper response code. Code is 404
730
Error improper response code. 

Error improper response code. Code is 404
889
Error improper response code. Code is 404
890
Error improper response code. Code is 404
891
Error improper response code. Code is 404
892
Error improper response code. Code is 404
893
Error improper response code. Code is 404
894
Error improper response code. Code is 404
895
Error improper response code. Code is 404
896
Error improper response code. Code is 404
897
Error improper response code. Code is 404
898
Error improper response code. Code is 404
899
Error improper response code. Code is 404
900
Error improper response code. Code is 404
901
Error improper response code. Code is 404
902
Error improper response code. Code is 404
903
Error improper response code. Code is 404
904
Error improper response code. Code is 404
905
Error improper response code. Code is 404
906
Error improper response code. Code is 404
907
Error improper response code. Code is 404
908
Error improper response code. Code is 404
909
Error improper response code. Code

Error improper response code. Code is 404
1066
Error improper response code. Code is 404
1067
Error improper response code. Code is 404
1068
Error improper response code. Code is 404
1069
Error improper response code. Code is 404
1070
Error improper response code. Code is 404
1071
Error improper response code. Code is 404
1072
Error improper response code. Code is 404
1073
Error improper response code. Code is 404
1074
Error improper response code. Code is 404
1075
Error improper response code. Code is 404
1076
Error improper response code. Code is 404
1077
Error improper response code. Code is 404
1078
Error improper response code. Code is 404
1079
Error improper response code. Code is 404
1080
Error improper response code. Code is 404
1081
Error improper response code. Code is 404
1082
Error improper response code. Code is 404
1083
Error improper response code. Code is 404
1084
Error improper response code. Code is 404
1085
Error improper response code. Code is 404
1086
Error imprope

Error improper response code. Code is 404
1241
Error improper response code. Code is 404
1242
Error improper response code. Code is 404
1243
Error improper response code. Code is 404
1244
Error improper response code. Code is 404
1245
Error improper response code. Code is 404
1246
Error improper response code. Code is 404
1247
Error improper response code. Code is 404
1248
Error improper response code. Code is 404
1249
Error improper response code. Code is 404
1250
Error improper response code. Code is 404
1251
Error improper response code. Code is 404
1252
Error improper response code. Code is 404
1253
Error improper response code. Code is 404
1254
Error improper response code. Code is 404
1255
Error improper response code. Code is 404
1256
Error improper response code. Code is 404
1257
Error improper response code. Code is 404
1258
Error improper response code. Code is 404
1259
Error improper response code. Code is 404
1260
Error improper response code. Code is 404
1261
Error imprope

In [68]:
len(list_for_mongo)

1226

### Create a mongoDB and have data stored from scrape 

In [69]:
# DB creation 
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
curlydb = myclient['Naturaly_Curly_db']
curly_collection = curlydb['curly_profiles']

In [70]:
entry2 = curly_collection.insert_many(list_for_mongo)

In [71]:
query_1 = curly_collection.find({})
test = []
for x in query_1:
    test.append(x['signature'])
    #print(x)
print(len(test))    

433709


In [72]:
test_set = set(test)

In [73]:
# Number of unique entries so far
len(test_set)
#7465

7492